In [1]:
!pip install -q datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.7 MB/s eta 0:00:00


In [2]:
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer,
    default_data_collator
)
import evaluate

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [3]:
dataset = load_dataset("taidng/UIT-ViQuAD2.0")

model_name = "vinai/phobert-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name).to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.04k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/5.20M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/735k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

...

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/540M [00:00<?, ?B/s]

In [4]:
dataset['train'][0]

{'id': '0001-0001-0001',
 'uit_id': 'uit_000001',
 'title': 'Phạm Văn Đồng',
 'context': 'Phạm Văn Đồng (1 tháng 3 năm 1906 – 29 tháng 4 năm 2000) là Thủ tướng đầu tiên của nước Cộng hòa Xã hội chủ nghĩa Việt Nam từ năm 1976 (từ năm 1981 gọi là Chủ tịch Hội đồng Bộ trưởng) cho đến khi nghỉ hưu năm 1987. Trước đó ông từng giữ chức vụ Thủ tướng Chính phủ Việt Nam Dân chủ Cộng hòa từ năm 1955 đến năm 1976. Ông là vị Thủ tướng Việt Nam tại vị lâu nhất (1955–1987). Ông là học trò, cộng sự của Chủ tịch Hồ Chí Minh. Ông có tên gọi thân mật là Tô, đây từng là bí danh của ông. Ông còn có tên gọi là Lâm Bá Kiệt khi làm Phó chủ nhiệm cơ quan Biện sự xứ tại Quế Lâm (Chủ nhiệm là Hồ Học Lãm).',
 'question': 'Tên gọi nào được Phạm Văn Đồng sử dụng khi làm Phó chủ nhiệm cơ quan Biện sự xứ tại Quế Lâm?',
 'answers': {'text': ['Lâm Bá Kiệt'], 'answer_start': [507]},
 'is_impossible': False,
 'plausible_answers': None}

In [5]:
dataset['train'][0]['answers']

{'text': ['Lâm Bá Kiệt'], 'answer_start': [507]}

In [6]:
max_length = 256

def compute_token_positions(context, answers):
    answer_text = answers["text"][0]

    context_ids = tokenizer.encode(context, add_special_tokens=False)
    answer_ids = tokenizer.encode(answer_text, add_special_tokens=False)

    if not answer_ids:
        return None, None

    for i in range(len(context_ids) - len(answer_ids) + 1):
        if context_ids[i: i + len(answer_ids)] == answer_ids:
            return i, i + len(answer_ids) - 1
    return None, None

def prepare_train_features(examples):
    outputs = {
        "input_ids": [],
        "attention_mask": [],
        "token_type_ids": [],
        "start_positions": [],
        "end_positions": []
    }
    for question, context, answers in zip(examples["question"], examples["context"], examples["answers"]):
        question_encoded = tokenizer.encode(question, add_special_tokens=True)

        encoded = tokenizer.encode_plus(
            question,
            context,
            truncation="only_second",
            max_length=max_length,
            padding="max_length"
        )

        if len(answers["answer_start"]) == 0:
            start_pos, end_pos = 0, 0
        else:
            token_start_index, token_end_index = compute_token_positions(context, answers)
            if token_start_index is None or token_end_index is None:
                start_pos, end_pos = 0, 0
            else:
                offset = len(question_encoded)
                start_pos = offset + token_start_index
                end_pos = offset + token_end_index

                if start_pos >= max_length or end_pos >= max_length:
                    start_pos, end_pos = 0, 0

        outputs["input_ids"].append(encoded["input_ids"])
        outputs["attention_mask"].append(encoded["attention_mask"])
        outputs["token_type_ids"].append(encoded["token_type_ids"])
        outputs["start_positions"].append(start_pos)
        outputs["end_positions"].append(end_pos)

    return outputs

In [7]:
def prepare_eval_features(examples):
    questions = examples["question"]
    contexts = examples["context"]

    tokenized_inputs = tokenizer(
        questions,
        contexts,
        max_length=max_length,
        truncation="only_second",
        padding="max_length",
    )

    return tokenized_inputs

In [8]:
train_dataset = dataset["train"].map(
    prepare_train_features,
    batched=True,
    remove_columns=dataset["train"].column_names,
)

Map:   0%|          | 0/28454 [00:00<?, ? examples/s]

In [9]:
eval_dataset = dataset["validation"].map(
    prepare_eval_features,
    batched=True
)

Map:   0%|          | 0/3814 [00:00<?, ? examples/s]

In [10]:
eval_dataset

Dataset({
    features: ['id', 'uit_id', 'title', 'context', 'question', 'answers', 'is_impossible', 'plausible_answers', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3814
})

In [11]:
# !git config --global credential.helper store

In [18]:
def compute_metrics(eval_pred):
    predictions, label_ids = eval_pred
    start_logits, end_logits = predictions
    pred_start = torch.argmax(start_logits, dim=1)
    pred_end = torch.argmax(end_logits, dim=1)

    predictions_list = []
    references_list = []
    for i, example in enumerate(eval_dataset):
        input_ids = example["input_ids"]
        question = example["question"]
        context = example["context"]
        answers = example["answers"]

        offset = len(tokenizer.encode(question, add_special_tokens=True))
        start_index = int(pred_start[i])
        end_index = int(pred_end[i])

        if start_index > end_index:
            pred_answer = ""
        if start_index < offset:
            pred_answer = ""
        else:
            pred_answer_ids = input_ids[start_index: end_index+1]
            pred_answer = tokenizer.decode(pred_answer_ids, skip_special_tokens=True)

        predictions_list.append({
            "id": example["id"],
            "prediction_text": pred_answer
        })
        references_list.append({
            "id": example["id"],
            "answers": answers
        })

    squad_metric = evaluate.load("squad")
    results = squad_metric.compute(predictions=predictions_list, references=references_list)
    return results

In [19]:
training_args = TrainingArguments(
    output_dir="./phobert_finetuned_viquad_results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
    push_to_hub=True,
    hub_strategy="every_save",
    hub_model_id="heidiie/phobert_finetuned_viquad",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
)

In [22]:
from huggingface_hub import notebook_login
notebook_login()

In [20]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [23]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [24]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: heidiie (heidiie-hus) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
1,1.050300,No log
2,0.761700,No log
3,0.614100,No log


TrainOutput(global_step=5337, training_loss=0.9181653016154543, metrics={'train_runtime': 1493.7361, 'train_samples_per_second': 57.147, 'train_steps_per_second': 3.573, 'total_flos': 1.1152406874249216e+16, 'train_loss': 0.9181653016154543, 'epoch': 3.0})

In [28]:
test_dataset = dataset["test"].map(
    prepare_eval_features,
    batched=True,
)

In [55]:
test_results = trainer.evaluate(eval_dataset=test_dataset)
print("Test Results:", test_results)

Test Results: {'eval_runtime': 28.2463, 'eval_samples_per_second': 258.477, 'eval_steps_per_second': 16.179, 'epoch': 3.0}


In [57]:
from transformers import pipeline

example_question = "Việt Nam có bao nhiêu tỉnh thành?"
example_context = "Việt Nam hiện có 63 tỉnh thành, bao gồm các thành phố trực thuộc trung ương và các tỉnh."
question_answerer = pipeline("question-answering", model="heidiie/phobert_finetuned_viquad", tokenizer=tokenizer)
question_answerer(question=example_question, context=example_context)

config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

Device set to use cuda:0


{'score': 0.22542990744113922, 'start': 17, 'end': 19, 'answer': '63'}